Coding Decoder-Only Transformers from Scratch

In [1]:
#Import required modules
import pip
try:
  __import__("lightning")
except ImportError:
  pip.main(['install', "lightning"])  # if lightning is not installed

import torch
import torch.nn as nn 
import torch.nn.functional as F

from torch.optim import Adam
from torch.utils.data import TensorDataset, DataLoader

import lightning as L

/usr/local/lib/python3.10/dist-packages/_distutils_hack/__init__.py:31: UserWarning: Setuptools is replacing distutils. Support for replacing an already imported distutils is deprecated. In the future, this condition will fail. Register concerns at https://github.com/pypa/setuptools/issues/new?template=distutils-deprecation.yml
  warnings.warn(
Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.


Collecting lightning

Downloading lightning-2.5.0.post0-py3-none-any.whl.metadata (40 kB)

Output()

Requirement already satisfied: PyYAML<8.0,>=5.4 in /usr/local/lib/python3.10/dist-packages (from lightning) (6.0.2)

...

Requirement already satisfied: intel-cmplr-lib-ur==2024.2.0 in /usr/local/lib/python3.10/dist-packages (from intel-openmp>=2024->mkl->numpy>1.20.0->torchmetrics<3.0,>=0.7.0->lightning) (2024.2.0)

Downloading lightning-2.5.0.post0-py3-none-any.whl (815 kB)

Output()

Installing collected packages: lightning

Successfully installed lightning-2.5.0.post0

Create input and output data:
Input: What is PyTorch?
Output: Interesting

In [2]:
# Create dictionary that maps vocabulary to numbers
token_to_id = {'what': 0,
              'is': 1,
              'PyTorch': 2,
              'Interesting': 3,
              '<EOS>': 4, #EOS is the end of sequence
              }

# Dictionary that maps ID to tokens to interpret output
id_to_token = dict(map(reversed, token_to_id.items()))
print(id_to_token)

#Inputs will contain the questions - "What is PyTorch?"", "PyTorch is what?" followed by response "Interesting"
inputs = torch.tensor([[token_to_id["what"], ## input #1: What is PyTorch <EOS> Intersting
                        token_to_id["is"], 
                        token_to_id["PyTorch"], 
                        token_to_id["<EOS>"],
                        token_to_id["Interesting"]], 
                       
                       [token_to_id["PyTorch"], # input #2: PyTorch is what <EOS> Intersting
                        token_to_id["is"], 
                        token_to_id["what"], 
                        token_to_id["<EOS>"], 
                        token_to_id["Interesting"]]])

labels = torch.tensor([[token_to_id["is"], 
                        token_to_id["PyTorch"], 
                        token_to_id["<EOS>"], 
                        token_to_id["Interesting"], 
                        token_to_id["<EOS>"]],  
                       
                       [token_to_id["is"], 
                        token_to_id["what"], 
                        token_to_id["<EOS>"], 
                        token_to_id["Interesting"], 
                        token_to_id["<EOS>"]]])

# Merge into a DataLoader
dataset = TensorDataset(inputs,labels)
dataloader = DataLoader(dataset)

{0: 'what', 1: 'is', 2: 'PyTorch', 3: 'Interesting', 4: '<EOS>'}


 Embedding and Positional Encoding
> NOTE: The reason why we are bothering to create a class to do positional encoding, instead of just adding this code directly to the transformer, is that we can easily re-use it in an Encoder-Only Transformer or an Encoder-Decoder Transformer. So, by creating a class that does positional encoding, we can code it once, and then just create instances when and where we need it.

> ALSO NOTE: Since the position encoding values never change, meaning that the first token always uses the same position encoding values regardless of what that token is, we precompute them and save them in a lookup table. This makes adding position encoding values super fast.

  

In [3]:
class PositionEncoding(nn.Module):
    def __init__(self, d_model=2, max_len=6):  #dmodel is the dimension of the transformer (No. of embeddings per token), max_len is the no. of tokens we allow as input
        super().__init__()
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(start=0, end=max_len, step=1).float().unsqueeze(1)
        embedding_index = torch.arange(start=0, end=d_model, step=2).float()
        
        div_term = 1/torch.tensor(10000.0)**(embedding_index / d_model)

        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)

        self.register_buffer('pe', pe)

    def forward(self, word_embeddings):
        return word_embeddings + self.pe[:word_embeddings.size(0), :]
        

Attention class

In [4]:
class Attention(nn.Module): 
    
    def __init__(self, d_model=2):
        super().__init__()
        self.d_model=d_model
        
        self.W_q = nn.Linear(in_features=d_model, out_features=d_model, bias=False)
        self.W_k = nn.Linear(in_features=d_model, out_features=d_model, bias=False)
        self.W_v = nn.Linear(in_features=d_model, out_features=d_model, bias=False)

        self.row_dim = 0
        self.col_dim = 1

        
    def forward(self, encodings_for_q, encodings_for_k, encodings_for_v, mask=None):
        q = self.W_q(encodings_for_q)
        k = self.W_k(encodings_for_k)
        v = self.W_v(encodings_for_v)

        sims = torch.matmul(q, k.transpose(dim0=self.row_dim, dim1=self.col_dim))

        scaled_sims = sims / torch.tensor(k.size(self.col_dim)**0.5)

        if mask is not None:
            scaled_sims = scaled_sims.masked_fill(mask=mask, value=-1e9)
        
        # Apply softmax to determine what percent of each token's value to use in the final attention values.
        attention_percents = F.softmax(scaled_sims, dim=self.col_dim)

        # Scale the values by their associated percentages and add them up.
        attention_scores = torch.matmul(attention_percents, v)
        
        return attention_scores

Decoder-Only Transformer

**A Decoder-Only Transformer simply brings together...**


* Word Embedding
* Position Encoding
* Masked Self-Attention
* Residual Connections
* A fully connected layer
* SoftMax - However, the loss function we are using nn.CrossEntropyLoss(), applies the SoftMax for us.


In [5]:
class DecoderOnlyTransformer(L.LightningModule):
    
    def __init__(self, num_tokens=4, d_model=2, max_len=6):
        super().__init__()
        L.seed_everything(seed=42)
        
        self.we = nn.Embedding(num_embeddings=num_tokens, embedding_dim=d_model)     
        
        self.pe = PositionEncoding(d_model=d_model, max_len=max_len)

        self.self_attention = Attention(d_model=d_model)

        self.fc_layer = nn.Linear(in_features=d_model, out_features=num_tokens)
        
        self.loss = nn.CrossEntropyLoss()
        
        
    def forward(self, token_ids):
                
        word_embeddings = self.we(token_ids)        
        position_encoded = self.pe(word_embeddings)

        mask = torch.tril(torch.ones((token_ids.size(dim=0), token_ids.size(dim=0)), device=self.device))
        mask = mask == 0
        
        self_attention_values = self.self_attention(position_encoded, 
                                                    position_encoded, 
                                                    position_encoded, 
                                                    mask=mask)
        
        residual_connection_values = position_encoded + self_attention_values
        
        fc_layer_output = self.fc_layer(residual_connection_values)
        
        return fc_layer_output
    
    
    def configure_optimizers(self): 
        return Adam(self.parameters(), lr=0.1)
    
    
    def training_step(self, batch, batch_idx): 
        input_tokens, labels = batch # collect input
        output = self.forward(input_tokens[0])
        loss = self.loss(output, labels[0])
                    
        return loss

Decoder-Only Transformer - Without Training

In [6]:
## First, create a model from DecoderOnlyTransformer()
model = DecoderOnlyTransformer(num_tokens=len(token_to_id), d_model=2, max_len=6)

model_input = torch.tensor([token_to_id["what"], 
                            token_to_id["is"], 
                            token_to_id["PyTorch"], 
                            token_to_id["<EOS>"]])
input_length = model_input.size(dim=0)

## predictions from the model
predictions = model(model_input) 
predicted_id = torch.tensor([torch.argmax(predictions[-1,:])])
predicted_ids = predicted_id

## use a loop to predict output tokens until we get an <EOS> token.
max_length = 6
for i in range(input_length, max_length):
    if (predicted_id == token_to_id["<EOS>"]): # if the prediction is <EOS>, then we are done
        break
    
    model_input = torch.cat((model_input, predicted_id))
    
    predictions = model(model_input) 
    predicted_id = torch.tensor([torch.argmax(predictions[-1,:])])
    predicted_ids = torch.cat((predicted_ids, predicted_id))
        
## Now printout the predicted output phrase.
print("Predicted Tokens:\n") 
for id in predicted_ids: 
    print("\t", id_to_token[id.item()])

INFO: Seed set to 42


Seed set to 42

Predicted Tokens:

	 <EOS>


Training the Decoder-Only Transformer

In [7]:
trainer = L.Trainer(max_epochs=30)
trainer.fit(model, train_dataloaders=dataloader)

model_input = torch.tensor([token_to_id["what"], 
                            token_to_id["is"], 
                            token_to_id["PyTorch"], 
                            token_to_id["<EOS>"]])
input_length = model_input.size(dim=0)

predictions = model(model_input) 
predicted_id = torch.tensor([torch.argmax(predictions[-1,:])])
predicted_ids = predicted_id

for i in range(input_length, max_length):
    if (predicted_id == token_to_id["<EOS>"]): # if the prediction is <EOS>, then we are done
        break
    
    model_input = torch.cat((model_input, predicted_id))
    
    predictions = model(model_input) 
    predicted_id = torch.tensor([torch.argmax(predictions[-1,:])])
    predicted_ids = torch.cat((predicted_ids, predicted_id))
        
print("Predicted Tokens:\n") 
for id in predicted_ids: 
    print("\t", id_to_token[id.item()])

INFO: GPU available: False, used: False


GPU available: False, used: False

INFO: TPU available: False, using: 0 TPU cores


TPU available: False, using: 0 TPU cores

INFO: HPU available: False, using: 0 HPUs


...

/usr/local/lib/python3.10/dist-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=3` in the `DataLoader` to improve performance.
/usr/local/lib/python3.10/dist-packages/lightning/pytorch/loops/fit_loop.py:310: The number of training batches (2) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.


Training: |          | 0/? [00:00<?, ?it/s]

INFO: `Trainer.fit` stopped: `max_epochs=30` reached.


`Trainer.fit` stopped: `max_epochs=30` reached.

Predicted Tokens:

	 Interesting
	 <EOS>


In [8]:
## Now let's ask the other question...
model_input = torch.tensor([token_to_id["PyTorch"], 
                            token_to_id["is"], 
                            token_to_id["what"], 
                            token_to_id["<EOS>"]])
input_length = model_input.size(dim=0)

predictions = model(model_input) 
predicted_id = torch.tensor([torch.argmax(predictions[-1,:])])
predicted_ids = predicted_id

for i in range(input_length, max_length):
    if (predicted_id == token_to_id["<EOS>"]): # if the prediction is <EOS>, then we are done
        break
    
    model_input = torch.cat((model_input, predicted_id))
    
    predictions = model(model_input) 
    predicted_id = torch.tensor([torch.argmax(predictions[-1,:])])
    predicted_ids = torch.cat((predicted_ids, predicted_id))
        
print("Predicted Tokens:\n") 
for id in predicted_ids: 
    print("\t", id_to_token[id.item()])

Predicted Tokens:

	 Interesting
	 <EOS>
